##### Copyright 2019 The TensorFlow Authors.

# Cham-GPT v0.1

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import os
import time

c:\Users\chamo\Miniconda\envs\girls_and_boys_in_tech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = '..\datasets\whatsapp_jasmeen_conversation.txt'

### Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 2488 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Cham: <Media omitted>
Jasrizz: That was amazing
Jasrizz: Who's my favourite super duper famous star??
Cham: oh jas, you're flattering me too much x
Jasrizz: And you're loving it x
Cham: yeah slightly
Cham: don't worry, it won't be long until yo


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

73 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[42, 43, 44, 45, 46, 47, 48], [65, 66, 67]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(2488,), dtype=int64, numpy=array([22, 49, 42, ..., 18,  2,  1], dtype=int64)>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

C
h
a
m
:
 
<
M
e
d


In [16]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'C' b'h' b'a' b'm' b':' b' ' b'<' b'M' b'e' b'd' b'i' b'a' b' ' b'o'
 b'm' b'i' b't' b't' b'e' b'd' b'>' b'\r' b'\n' b'J' b'a' b's' b'r' b'i'
 b'z' b'z' b':' b' ' b'T' b'h' b'a' b't' b' ' b'w' b'a' b's' b' ' b'a'
 b'm' b'a' b'z' b'i' b'n' b'g' b'\r' b'\n' b'J' b'a' b's' b'r' b'i' b'z'
 b'z' b':' b' ' b'W' b'h' b'o' b"'" b's' b' ' b'm' b'y' b' ' b'f' b'a'
 b'v' b'o' b'u' b'r' b'i' b't' b'e' b' ' b's' b'u' b'p' b'e' b'r' b' '
 b'd' b'u' b'p' b'e' b'r' b' ' b'f' b'a' b'm' b'o' b'u' b's' b' ' b's'
 b't' b'a' b'r'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Cham: <Media omitted>\r\nJasrizz: That was amazing\r\nJasrizz: Who's my favourite super duper famous star"
b"??\r\nCham: oh jas, you're flattering me too much x\r\nJasrizz: And you're loving it x\r\nCham: yeah slight"
b"ly\r\nCham: don't worry, it won't be long until you get your own private show x\r\nJasrizz: That sounds v"
b'aguely sexual, you  better strip for me \xf0\x9f\x91\x80\r\nCham: it unfortunately sounded very sexual\r\nCham: but I do'
b" not disappoint\r\nJasrizz: Better hurry back to Nottingham sooner then later then\r\nCham: oh I know, I'"


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Cham: <Media omitted>\r\nJasrizz: That was amazing\r\nJasrizz: Who's my favourite super duper famous sta"
Target: b"ham: <Media omitted>\r\nJasrizz: That was amazing\r\nJasrizz: Who's my favourite super duper famous star"


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [23]:
# Batch size
BATCH_SIZE = 5

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(5, 100), dtype=tf.int64, name=None), TensorSpec(shape=(5, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(5, 100, 74) # (batch_size, sequence_length, vocab_size)


In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch)

tf.Tensor(
[[61 56  3 44 56 54 46  3 61 56 48 46 61 49 46 59  3 47 56 59  3 61 49 50
  60  3 46 63 46 55 61 10  3 37 49 50 60  3 64 50 53 53  3 50 55 44 53 62
  45 46 16  2  1 60 60  3  2  1 50 55 48  3  2  1 60 60  2  1 42 60 60  2
   1  2  1  2  1  2  1  2  1 33  3 33 34 40  8  2  1 22 49 42 54 16  3 17
  32 46 45 50]
 [37 49 56 10 10 10  3 28  3 45 50 45  3 61 42 52 46  3 50 61  3 56 47 47
   3 66 56 62 59  3 64 42 50 60 61  2  1 22 49 42 54 16  3 46 65 42 44 61
  53 66  9  3 28  5 54  3 42 55  3 50 55 55 56 44 46 55 61  3 63 50 44 61
  50 54  3 72  2  1 29 42 60 59 50 67 67 16  3 41 56 62  3 53 56 63 46 45
   3 46 63 46]
 [ 1 62 57 46 59 43  3 13 11 13 14  3 68 69  3 64 50 61 49  3 60 56 54 46
   3 50 55 44 59 46 45 50 43 53 46  3 42 44 49 50 46 63 46 54 46 55 61 60
   3 64 46  3 64 56 62 53 45  3 53 56 63 46  3 73  3 42 60  3 54 42 55 66
   3 60 61 62 45 46 55 61 60  3 42 55 45  3 47 42 54 50 53 66  3 54 46 54
  43 46 59 60]
 [67 67 16  3 20 53 60 56  3 61 49 42 55 52  3 66 56 62 

In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  18944     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  75850     
                                                                 
Total params: 4033098 (15.39 MB)
Trainable params: 4033098 (15.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([58, 12, 58, 11, 28, 25, 58, 44, 62, 59,  8, 30, 50, 38, 31, 56, 22,
       17, 16, 10, 19, 26, 23,  0, 10, 12, 72, 24, 36, 18, 11, 30, 46,  7,
       73, 72, 36, 21, 14, 54, 41, 49, 68, 54, 24, 25, 55, 60, 19, 53, 15,
       70, 69,  3, 64, 49, 15, 17, 45, 56, 60, 38, 72, 64, 51, 65, 42, 11,
       25,  2, 43, 22, 10, 37, 33, 42, 12, 72, 20, 68, 70, 60, 31, 24, 40,
       15, 30, 67, 45, 42,  0, 63,  8,  4, 37, 55, 57, 14, 69, 28],
      dtype=int64)

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'to come together for this event. This will include:\r\nss \r\ning \r\nss\r\nass\r\n\r\n\r\n\r\n\r\nN NOW*\r\nCham: <Medi'

Next Char Predictions:
 b'q1q0IFqcur*KiULoC<:.?GD[UNK].1\xf0\x9f\x98\xadES>0Ke)\xf0\x9f\xa5\xb0\xf0\x9f\x98\xadSB3mYh\xe2\x9c\xa8mEFns?l6\xf0\x9f\x91\x80\xef\xb8\x8f wh6<dosU\xf0\x9f\x98\xadwjxa0F\rbC.TNa1\xf0\x9f\x98\xadA\xe2\x9c\xa8\xf0\x9f\x91\x80sLEW6Kzda[UNK]v*&Tnp3\xef\xb8\x8fI'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (5, 100, 74)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.303265, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

73.94083

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [36]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [51]:
EPOCHS = 5

history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/5


4/4 [==============================] - 4s 966ms/step - loss: 2.8805
Epoch 2/5
4/4 [==============================] - 4s 957ms/step - loss: 2.7425
Epoch 3/5
4/4 [==============================] - 4s 956ms/step - loss: 2.5946
Epoch 4/5
4/4 [==============================] - 4s 949ms/step - loss: 2.4554
Epoch 5/5
4/4 [==============================] - 4s 952ms/step - loss: 2.2772


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [55]:
user_input = 'Hello'

start = time.time()
states = None
next_char = tf.constant(['Dylan: '+str(user_input)+'\nCham: '])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Dylan: Hello
Cas brinz tede har <ll y
ham: <nng>
Jasmz: <ud 

CJam: (niudrer
Cham vde sam foiudr
 

________________________________________________________________________________

Run time: 0.28129148483276367


The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO: ca\nm.J<:>twA\ry?K&uHIo) )b2L3lu6Y<cDuII<BTt<\n\r2kKz:Avr\xf0\x9f\x98\xadJ?\r>S,rc6?0:t\ngL6 D16z\xf0\x9f\x98\x88\xf0\x9f\xa5\xb0?3>6yzzL*mJCiDaEd?w\xef\xb8\x8f)rDa2e>.Vj?(cnasiR6&Uwj:eUTO6RosNd\xe2\x9c\xa8JCifufVvbt\n>rT\r2F\xf0\x9f\x98\x88r.3xii1Wh:3(AK<rCBEG>lCEEL\nCo)RG<3H\xe2\x9c\xa8F(Dr)&OG,JTarOx\r LohnSg\xf0\x9f\x98\x88uI3(\xef\xb8\x8fb\xf0\x9f\xa5\xb0t,A)f,D0\xf0\x9f\x98\x88zDf:t&Id\r:?>,b(xv::mgl:By\xf0\x9f\x98\x88d)cyfh\xf0\x9f\xa5\xb0\rssz.\xe2\x9c\xa8sd?nEe\rx\r,Yd:nRde?td<pr\xef\xb8\x8fGW'zJA2Cya\xf0\x9f\x91\x80SOsRFuTH orf)aet\nhI(ba:tn\xf0\x9f\x98\x88(<JmNlt **:'iLW(b6ke\xf0\x9f\x98\x88yWcT>xcl2\xf0\x9f\xa5\xb0IYkvRDtrJvy'>tyzaxlL fy2rhKIm6AOfCq0'.,aR\r<n\r.TeMnW\xf0\x9f\x98\xadKjTd\xef\xb8\x8fu1khB1U\xe2\x9c\xa8kf3u\ro\xf0\x9f\x91\x80(d.C,2taO?se:p\xf0\x9f\xa5\xb0f1q6n\xef\xb8\x8f0'.IDp\xf0\x9f\xa5\xb0aE&dkx:r?'NAgVf0j'\r,gWt6g>U\xf0\x9f\x98\xademqb\xe2\x9c\xa8.SLuf.\xf0\x9f\x91\x80**,1gzHYty\n\xf0\x9f\xa5\xb0v\rhM)\xe2\x9c\xa8'juz\r2GSBMSrrsAi\xef

In [43]:
print(1/0)

ZeroDivisionError: division by zero

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
While shall we toward them, gaunt and married. Urped me say I
doubt not, for this world is gentle, 


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)


  1/172 [..............................] - ETA: 10:00 - loss: 4.1898


  2/172 [..............................] - ETA: 16s - loss: 4.1720  


  3/172 [..............................] - ETA: 16s - loss: 4.1475


  4/172 [..............................] - ETA: 13s - loss: 4.0987


  6/172 [>.............................] - ETA: 11s - loss: 4.3665

...


169/172 [============================>.] - ETA: 0s - loss: 2.7202


170/172 [============================>.] - ETA: 0s - loss: 2.7170


171/172 [============================>.] - ETA: 0s - loss: 2.7139


172/172 [==============================] - ETA: 0s - loss: 2.7107


172/172 [==============================] - 13s 58ms/step - loss: 2.7075


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1913


Epoch 1 Batch 50 Loss 2.0591


Epoch 1 Batch 100 Loss 1.9363


Epoch 1 Batch 150 Loss 1.8937



Epoch 1 Loss: 1.9814
Time taken for 1 epoch 12.44 sec
________________________________________________________________________________


...

Epoch 10 Batch 0 Loss 1.1429


Epoch 10 Batch 50 Loss 1.1642


Epoch 10 Batch 100 Loss 1.1455


Epoch 10 Batch 150 Loss 1.1687



Epoch 10 Loss: 1.1684
Time taken for 1 epoch 11.55 sec
________________________________________________________________________________
